In [ ]:
# @title Launch ImageDream
# @markdown Connect to A100 GPU before running.
import os
import shutil
import sys
from google.colab import drive, files

# Paths in your Google Drive
weights_path = "/content/drive/MyDrive/ImageDream_Colab_Backup/sd-v2.1-base-4view-ipmv-local.pt"
installer_folder = "/content/drive/MyDrive/Colab_Installers"
# -----------------------------

In [ ]:
# 2. Setup Environment
drive.mount('/content/drive')

print("\n Installing optimized libraries...")
try:
    # A. Install tiny-cuda-nn from Drive Cache
    wheel_files = [f for f in os.listdir(installer_folder) if f.endswith(".whl")]
    if not wheel_files:
        print(f"  Warning: No wheel found in {installer_folder}.")
    else:
        wheel_path = os.path.join(installer_folder, wheel_files[0])
        print(f"   Installing from: {wheel_path}")
        !pip install "{wheel_path}" ninja imageio[ffmpeg]

    # B. Clone YOUR Fork
    print(f"\n  Cloning https://github.com/erikhado/ImageDream.git...")
    %cd /content
    if os.path.exists("ImageDream"):
        shutil.rmtree("ImageDream")
    !git clone https://github.com/erikhado/ImageDream.git

    # C. Install Requirements
    print("\n  Installing Requirements...")
    %cd /content/ImageDream
    !pip install -r requirements.txt

    # --- D. The Nested Folder Fix ---
    # We look for the nested ImageDream folder to install the package properly
    nested_root = "/content/ImageDream/extern/ImageDream"
    if os.path.exists(nested_root):
        print(f"\n  Found nested source at {nested_root}. Switching context...")
        %cd {nested_root}
    else:
        print("\n  Standard structure detected (no nested extern/ImageDream).")
        nested_root = "/content/ImageDream" # Fallback

    # Create the missing setup.py to force installation
    setup_content = """
from setuptools import setup, find_packages
setup(name="imagedream", version="1.0.0", packages=find_packages())
"""
    with open("setup.py", "w") as f:
        f.write(setup_content)

    !pip install -e .
    !pip install libigl==2.5.1

    # E. Setup MVDream (Forked)
    print("\n  Setting up MVDream...")
    # Go back to the real project root to place extern/MVDream correctly
    %cd /content/ImageDream
    !mkdir -p extern
    %cd extern
    # Clone your forked MVDream here
    !git clone https://github.com/erikhado/MVDream.git MVDream
    %cd MVDream
    !pip install -e .

    print("\n✅ Environment Ready!")
except Exception as e:
    print(f"\n❌ Error during setup: {e}")

In [ ]:
# 3. Link Weights
print("\n  Linking weights...")
target_weights = os.path.join(nested_root, "release_models/ImageDream/sd-v2.1-base-4view-ipmv.pt")

if not os.path.exists(target_weights):
    os.makedirs(os.path.dirname(target_weights), exist_ok=True)
    if os.path.exists(weights_path):
        os.symlink(weights_path, target_weights)
        print(f"   Linked {weights_path} -> {target_weights}")
    else:
        print(f"❌ Error: Weights file not found at {weights_path}")

In [ ]:
# 4. Upload Image
print("\n---  Upload Image ---")
assets_dir = "/content/ImageDream/assets"
os.makedirs(assets_dir, exist_ok=True)
for f in os.listdir(assets_dir): os.remove(os.path.join(assets_dir, f)) # Clean old

uploaded = files.upload()
filename = list(uploaded.keys())[0]
shutil.move(filename, os.path.join(assets_dir, filename))
image_path = os.path.join(assets_dir, filename)

In [ ]:
%cd /content/ImageDream/extern/ImageDream/release_models/ImageDream/
!ls

In [ ]:
# @title Final Launch Command
import os

# Define absolute paths based on the current context
project_root = "/content/ImageDream"
mvdream_path = os.path.join(project_root, "extern", "MVDream")
deep_source_root = "/content/ImageDream/extern/ImageDream" # The root of your source package

# --- ABSOLUTE PATHS ARE DEFINED HERE ---
ckpt_path_absolute = "/content/ImageDream/extern/ImageDream/release_models/ImageDream/sd-v2.1-base-4view-ipmv.pt"

# FIX: Define the guidance config path using the absolute path to the nested source code
guidance_config_absolute = os.path.join(deep_source_root, 'imagedream/configs/sd_v2_base_ipmv.yaml')
# ---------------------------------------

train_config = "configs/imagedream-sd21-shading.yaml"
image_path = os.path.join(project_root, "assets", "astronaut.png")

# Set PYTHONPATH and move to directory
env_vars = f"PYTHONPATH='{project_root}:{mvdream_path}:$PYTHONPATH'"
%cd {project_root}

print(" Launching...")

!{env_vars} python launch.py \
    --config {train_config} \
    --train \
    --gpu 0 \
    name="imagedream_final" \
    tag="fully_fixed" \
    system.prompt_processor.prompt="3d asset" \
    system.prompt_processor.image_path="{image_path}" \
    system.guidance.ckpt_path="{ckpt_path_absolute}" \
    system.guidance.config_path="{guidance_config_absolute}"


In [ ]:
# @title Zip & Download Full Run
import shutil
import os
from google.colab import files

# --- Configuration ---
project_root = "/content/ImageDream"
output_base = os.path.join(project_root, "outputs", "imagedream_final")

try:
    # 1. Find the latest run folder (e.g., fully_fixed@2025...)
    # This is better than zipping the whole 'outputs' if you have multiple failed attempts
    runs = [os.path.join(output_base, d) for d in os.listdir(output_base) if os.path.isdir(os.path.join(output_base, d))]
    latest_run = max(runs, key=os.path.getmtime)

    print(f"✅ Found latest run at: {latest_run}")

    # 2. Define zip filename
    zip_name = "imagedream_full_run"
    zip_path = f"/content/{zip_name}"

    # 3. Create the Zip
    print(f"⏳ Zipping {latest_run} ... (This handles the large 1GB+ checkpoint)")
    shutil.make_archive(zip_path, 'zip', latest_run)

    # 4. Trigger Download
    final_zip = f"{zip_path}.zip"
    print(f"✅ Zip created: {final_zip}")
    print("⬇️ Downloading now...")
    files.download(final_zip)

except Exception as e:
    print(f"❌ Error: {e}")
    print("Could not find the output directory. Did you run the training?")